In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
from concurrent.futures import ThreadPoolExecutor

In [2]:
from langgraph.graph import StateGraph, END
from typing import Annotated, TypedDict, List
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage, AnyMessage
import operator

In [3]:
model = AzureChatOpenAI(openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt4chat"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://gpt-4-trails.openai.azure.com/"),
    api_key=os.environ.get("AZURE_OPENAI_KEY"))

In [4]:
from tavily import TavilyClient
tavily = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

In [5]:
class AgentState(TypedDict):
    Current_location: str # This is current user locatation
    Interests: str # This gives more idea about where user would love to travel like
    # beaches, mountains , serene places, spirutuality etc
    Budget: str # How much is the budget for this trip
    Domestic_or_International: str
    Accomodations_preference: str 
    Transportation_preference: str
    user_preference: str # This is the plan generated by the plan agent
    destinations: List[str] # These are the list of destinations Tavily has returned based on the user preference
    accomodations: List[str] # These are the list of hotels/ hostels Tavily has returned based on destinations and budget
    transportation: str # This is the list of documents that tavily has returned based on destinations and budget
    draft: str # This is the final internary and has all the details
    final_draft: str # This is the final output

In [6]:
from langchain_core.pydantic_v1 import BaseModel

class Queries(BaseModel):
    queries: List[str]

In [7]:
Preference_Analyzer_Prompt = """
You are an expert Trip Advisor. \
Your role is to understand the user's preference \
based on their interests, and travel budget \
Your job is to summarize the user preferences only.
"""

In [8]:
def Preference_Analyzer(state:AgentState):
    messages = [
        SystemMessage(content=Preference_Analyzer_Prompt),
        HumanMessage(content= "Here is my budget: " + state['Budget'] + "These are my interests: "
                     + state['Interests'] + "Here is my current location: "+ state['Current_location'] +
                      "I would love to travel: " + state['Domestic_or_International']),
    ]
    response = model.invoke(messages)
    return {"user_preference": response.content}

In [9]:
Destination_Researcher_Prompt = """ You are an expert in choosing perfect \
locations to travel based on user preferences \
Generate 3 queries at max to search and gather relevant information
and choose only 3 best places.
"""

In [10]:
def Destination_Researcher(state:AgentState):
    queries = model.with_structured_output(Queries).invoke([
            SystemMessage(content=Destination_Researcher_Prompt),
            HumanMessage(content= state['user_preference']),
        ])
    destinations = state['destinations'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results = 3)
        for r in response['results']:
            destinations.append(r['content'])
    return {"destinations": destinations}

In [11]:
Accommodation_Finder_Prompt = """ You are an expert in finding \
great accommodations based on the user preference. \
Generate a query each based on the number of destinations that will gather all the best accommodations based on the \
user preference and the destinations.
Utilize all the destinations information below as needed

{content}
"""

In [12]:
def Accommodation_Finder(state: AgentState):
    content = "\n\n".join(state['destinations'] or [])
    user_message = HumanMessage(
        content = f"{state['user_preference']}\n\n Here is my accommodation preference:\n\n{state['accomodations']}"
    )
    system_message = SystemMessage(content=Accommodation_Finder_Prompt.format(content=content))
    queries = model.with_structured_output(Queries).invoke([
            system_message,
            user_message
        ])
    accomodations = state['accomodations'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results = 3)
        for r in response['results']:
            accomodations.append(r['content'])
    return { "accomodations": accomodations}

In [13]:
Transportation_Optimizer_Prompt = """ You are great at helping travelers find the \
Best way to travel around their destinations and also keep in mind the accommodations, \
Make sure the travel is easy. and 
Genereate 2 queries max and Utilize all the destinations information below as needed 

{content}
"""

In [14]:
def Transportation_Optimizer(state: AgentState):
    content = "\n\n".join(state['accomodations'] or []) + "\n\n".join(state['destinations'] or [])
    user_message = HumanMessage(
        content = f"{state['user_preference']}\n\n Here is my Transportation preference:\n\n{state['Transportation_preference']}"
    )
    system_message = SystemMessage(content=Transportation_Optimizer_Prompt.format(content=content))
    queries = model.with_structured_output(Queries).invoke([
            system_message,
            user_message
        ])
    for q in queries.queries:
        response = tavily.search(query=q, max_results = 3)
    return { "transportation": response.content}

In [15]:
Itinerary_Builder_and_Culture_Advisor_Prompt = """ \
Create engaging day-by-day travel plans:

Design detailed itineraries that balance popular attractions with hidden gems.
Include a mix of activities catering to the traveler's interests (e.g., historical sites, nature, food experiences, art).
Consider travel time between locations and suggest efficient routes.
Recommend local events or seasonal activities happening during the travel dates.


Provide cultural insights and etiquette guidance:

Explain important local customs, traditions, and social norms.
Offer advice on appropriate dress codes for different settings (e.g., religious sites, restaurants).
Suggest polite phrases in the local language and explain their significance.
Highlight any taboos or behaviors to avoid that might offend locals.
Recommend ways for travelers to respectfully engage with the local culture

Generate 2 queries at max to gather all the necessary information, utilize the below information to form queries.
{content}

"""

In [16]:
def Itinerary_Builder_and_Culture_Advisor(state: AgentState):
    content = "\n\n".join(state['accomodations'] or []) + "\n\n".join(state['destinations'] or [])
    user_message = HumanMessage(
        content = f"{state['user_preference']}"
    )
    system_message = SystemMessage(content=Transportation_Optimizer_Prompt.format(content=content))
    queries = model.with_structured_output(Queries).invoke([
            system_message,
            user_message
        ])
    for q in queries.queries:
        response = tavily.search(query=q, max_results = 3)
    return {"draft": response.content}

In [17]:
Budget_Manager_Prompt = """ You are great at managing budget during travel\
and you know where to spend and where not to. can you please provide the break down of \
the budget in a human readable format

You can use all the context provided below

{content}
"""

In [18]:
def Budget_Manager(state: AgentState):
    content = "\n\n".join(state['accomodations'] or []) + "\n\n".join(state['destinations'] or []
        + "\n\n".join(state['transportation']) + "\n\n".join(state['draft']))
    user_message = HumanMessage(
        content = f"Give me my Trip Budget Breakdown total budget {state['Budget']}"
    )
    system_message = SystemMessage(content=Budget_Manager_Prompt.format(content=content))
    messages = [system_message, user_message]
    response = model.invoke(messages)
    draft = state['draft']+ response.content
    return {"draft": draft}

In [19]:
Drafting_Agent_Prompt = """ You have got a great knack for designing please use \
below format to give final draft

Example:-
Iternary
Day 1: Arrival and City Tour

	* Morning:
	•	Arrival at the hotel
	•	Check-in and freshen up
	•	Breakfast at the hotel
	* Afternoon:
	•	City tour
	•	Visit the main landmarks (e.g., central square, historic buildings)
	•	Lunch at a local restaurant
	* Evening:
	•	Dinner at a fine dining restaurant
	•	Free time to explore the local nightlife

example
Budget:-
1) Travel via plane:- 200$
2) Eating out:- 300$
.
.
.
n) Miscellinious:- 100$

Use the below information, you can be creative but \
do not add something that you do not have infromation about
{content}
"""

In [20]:
def Design_agent(state: AgentState):
    content = "\n\n ".join(state['draft'])
    user_message = HumanMessage(
        content = f"Please give me the final draft. Thank you"
    )
    system_message = SystemMessage(content=Drafting_Agent_Prompt.format(content=content))
    messages = [system_message, user_message]
    response = model.invoke(messages)
    return {"final_draft": response.content}

In [21]:
builder = StateGraph(AgentState)

In [22]:
builder.add_node("User_guide", Preference_Analyzer)
builder.add_node("Destination_researcher", Destination_Researcher)
builder.add_node("Accommodation_agent", Accommodation_Finder)
builder.add_node("Transportation_agent", Transportation_Optimizer)
builder.add_node("Itinerary_agent", Itinerary_Builder_and_Culture_Advisor)
builder.add_node("Budget_manager", Budget_Manager)
builder.add_node("Final_draft", Design_agent)

In [23]:
builder.set_entry_point("User_guide")
builder.set_finish_point("Final_draft")

In [24]:
# def should_continue(state: AgentState):
#     if state['final_draft']:
#         return END

In [25]:
builder.add_edge("User_guide", "Destination_researcher")
builder.add_edge("Destination_researcher", "Accommodation_agent")
builder.add_edge("Accommodation_agent", "Transportation_agent")
builder.add_edge("Transportation_agent", "Itinerary_agent")
builder.add_edge("Itinerary_agent", "Budget_manager")
builder.add_edge("Budget_manager", "Final_draft")

In [26]:
graph = builder.compile()

In [27]:
for s in graph.stream({
    "Current_location": "United States",
    "Interests": "Love Beaches and Mountains and also Treking",
    "Budget": "5000$ should be lower than the mentioned amount",
    "Domestic_or_International": "Any, But would love to travel internationally if possible",
    "Accomodations_preference": "Closer to attractions",
    "Transportation_preference": "Walk, can take public transport or a taxi",
}):
    print(s)

{'User_guide': {'user_preference': 'The user has a budget of $5000 and is interested in beaches, mountains, and trekking. They are currently located in the United States and are open to any travel destination, but would prefer to travel internationally if possible.'}}
{'Destination_researcher': {'destinations': ['The North Beach is a great place to watch birds and ships; Mid Beach is the spot to hunt for seashells and shark teeth; South Beach is the busiest area with volleyball and dolphin-spotting; and Back River is a more secluded beach experience with a fishing pier and great views of Little Tybee, the nature preserve just south of the barrier island. Spend time walking around the pastel buildings in the old town center, marveling at the ruins of the faded mansions of the Pietermaai neighborhood, and take a day trip to Klein (or, “little”) Curaçao, a deserted island with an old lighthouse, shipwrecks, and ancient stone buildings to explore.\nStop by a casual restaurant for affordabl

AttributeError: 'dict' object has no attribute 'content'